In [58]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc
nlp = spacy.load("ru_core_news_sm")


In [7]:
with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)
discription_of_event


{'История': {'знать': ['основные положения теории истории',
   'основные положения методологии истории',
   'место истории в системе гуманитарного знания',
   'закономерности исторического процесса',
   'этапы исторического процесса',
   'основные события истории России',
   'основные процессы истории России',
   'основные источники исторического знания',
   'историческую обусловленность современных общественно-политических процессов',
   'взаимосвязь истории России и мира',
   'особенности истории России и мира'],
  'уметь': ['формулировать свою позицию по вопросам, касающимся отношения к историческому прошлому',
   'обосновывать свою позицию по вопросам, касающимся отношения к историческому прошлому',
   'проводить комплексный поиск исторической информации в источниках различного типа',
   'устанавливать причинно-следственные связи между явлениями',
   'устанавливать пространственные рамки исторических процессов и явлений',
   'устанавливать временные рамки исторических процессов и я

In [50]:
kcm = {'знать': [], 'уметь': [], 'владеть': []}
for i in discription_of_event.values():
    for j in i:
        for z in i[j]:
            kcm[j].append(list(nlp(z).vector))
kcm

{'знать': [[0.9719262,
   -0.24668853,
   -0.18388611,
   -0.13750443,
   0.70293176,
   -0.08061214,
   2.0313377,
   0.20080651,
   -0.23104917,
   -0.34015533,
   -0.4963366,
   -0.4845267,
   0.2118981,
   1.6791135,
   -0.19480963,
   0.54853415,
   -0.31256518,
   0.044573754,
   -1.1142507,
   0.66456294,
   0.15482736,
   -1.1972735,
   1.4632698,
   0.5080032,
   -0.7042742,
   0.2282288,
   0.39757556,
   -0.6746778,
   0.95496356,
   -0.50043863,
   0.37583262,
   -0.20071527,
   -0.33813244,
   -0.24957621,
   0.42097726,
   0.91260004,
   -1.768919,
   -0.84987766,
   0.14220063,
   -0.12467447,
   -0.4737767,
   -0.16319917,
   0.2692668,
   -0.704191,
   -0.32524067,
   -1.4830095,
   -0.21121973,
   -0.1844316,
   -1.4933989,
   -0.44397372,
   -0.79498184,
   -1.0633922,
   0.283873,
   -0.80817175,
   0.83054376,
   0.21715409,
   -2.1662505,
   0.50893974,
   -0.6740733,
   0.1521059,
   -0.13575965,
   0.3178509,
   -1.2385302,
   -0.2861325,
   0.01609081,
   0.267

In [51]:
len(kcm.values()), len(kcm['знать']), len(kcm['знать'][0])


(3, 228, 96)

In [52]:
k = []
for i in kcm:
    for j in kcm[i]:
        k.append(j+[i])
# pd.DataFrame(k, columns=range(len(k[0])))
# sorted(k, key=len, reverse=False)
k

[[0.9719262,
  -0.24668853,
  -0.18388611,
  -0.13750443,
  0.70293176,
  -0.08061214,
  2.0313377,
  0.20080651,
  -0.23104917,
  -0.34015533,
  -0.4963366,
  -0.4845267,
  0.2118981,
  1.6791135,
  -0.19480963,
  0.54853415,
  -0.31256518,
  0.044573754,
  -1.1142507,
  0.66456294,
  0.15482736,
  -1.1972735,
  1.4632698,
  0.5080032,
  -0.7042742,
  0.2282288,
  0.39757556,
  -0.6746778,
  0.95496356,
  -0.50043863,
  0.37583262,
  -0.20071527,
  -0.33813244,
  -0.24957621,
  0.42097726,
  0.91260004,
  -1.768919,
  -0.84987766,
  0.14220063,
  -0.12467447,
  -0.4737767,
  -0.16319917,
  0.2692668,
  -0.704191,
  -0.32524067,
  -1.4830095,
  -0.21121973,
  -0.1844316,
  -1.4933989,
  -0.44397372,
  -0.79498184,
  -1.0633922,
  0.283873,
  -0.80817175,
  0.83054376,
  0.21715409,
  -2.1662505,
  0.50893974,
  -0.6740733,
  0.1521059,
  -0.13575965,
  0.3178509,
  -1.2385302,
  -0.2861325,
  0.01609081,
  0.26712596,
  -0.6402625,
  -0.28742015,
  0.97494775,
  -0.5154019,
  1.8061972

In [53]:
len(k), len(k[0])

(707, 97)

In [56]:
data = pd.DataFrame(k, columns=range(len(k[0])))

In [57]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(
    96, axis=1), data[96], test_size=0.25, random_state=42)


# Bayes

# Подбираем гиппер-параметры.

In [59]:
gaussian_naive_bayes_optimal_n_neighbors = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=accuracy_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_n_neighbors.best_params_


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\utils\_param_validation.py", line 175, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "C:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\inspect.py", line 3062, in bind
    return self._bind(args, kwargs)
  File "C:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\inspect.py", line 2988, in _bind
    raise TypeError(
TypeError: too many positional arguments

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\m

{'var_smoothing': 0.0}

# Формула Байеса

In [60]:
gaussian_naive_bayes = GaussianNB(
    var_smoothing=gaussian_naive_bayes_optimal_n_neighbors.best_params_['var_smoothing'])
gaussian_naive_bayes.fit(X_train, y_train)
y_predicted_by_gaussian_naive_bayes = gaussian_naive_bayes.predict(X_test)


# Оценка качества 

In [62]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes))



Accuracy
0.903954802259887
Confusion matrix
[[39  2  2]
 [ 7 55  3]
 [ 3  0 66]]
Precision, Recall, F
               precision    recall  f1-score   support

     владеть       0.80      0.91      0.85        43
       знать       0.96      0.85      0.90        65
       уметь       0.93      0.96      0.94        69

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.91      0.90      0.90       177



In [ ]:
fpr, tpr, thresholds = roc_curve(y_predicted_by_gaussian_naive_bayes, y_test)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(
    fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='example estimator')
display.plot()
